In [1]:
import simpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Define zero-rate periods (in minutes from the start of the day)
zero_rate_periods = [
    (20 * 60, 20.5 * 60),  # 20:00 - 20:30
    (0 * 60, 0.25 * 60),   # 00:00 - 00:15
    (3 * 60, 6.5 * 60),    # 03:00 - 06:30
    (9.5 * 60, 10 * 60),   # 09:30 - 10:00
    (2.5 * 60, 2.75 * 60), # 02:30 - 02:45
    (16 * 60, 18 * 60)     # 16:00 - 18:00
]

def in_zero_rate_periods(time):
    """Check if the given time falls within any zero-rate periods."""
    for start, end in zero_rate_periods:
        if start <= time % (24 * 60) <= end:
            return True
    return False

def manufacturing_process(env, name, mean_time, std_dev, data, package_id, previous_station_event=None):
    if previous_station_event:
        yield previous_station_event  # Wait for the previous station to complete

    start_time = env.now
    # Simulate processing with pauses during zero-rate periods
    while in_zero_rate_periods(start_time):
        yield env.timeout(1)  # Wait for 1 minute if within zero-rate period
        start_time += 1

    process_time = random.gauss(mean_time, std_dev)  # Generate process time with normal distribution
    yield env.timeout(process_time)  # Simulate processing time
    end_time = env.now
    data.append({'package_id': package_id, 'station': name, 'start_time': start_time, 'end_time': end_time, 'duration': process_time})
    print(f'Package {package_id} at {name} finished at {env.now}')
    return env.event().succeed()

def arrival_process(env, demand_data, mean_times, std_devs, data):
    for idx, row in demand_data.iterrows():
        timestamp = row['timestamp']
        num_packages = row['num_packages']
        
        yield env.timeout(timestamp - env.now)  # Wait until the arrival time

        for i in range(num_packages):
            package_id = f'Package {idx}-{i}'
            station_1_event = env.process(manufacturing_process(env, 'Station 1', mean_times[0], std_devs[0], data, package_id))

            # Stations 2 and 3
            station_2_event = env.process(manufacturing_process(env, 'Station 2', mean_times[1], std_devs[1], data, package_id, station_1_event))
            station_3_event = env.process(manufacturing_process(env, 'Station 3', mean_times[2], std_devs[2], data, package_id, station_1_event))

            # New Parallel Station
            station_parallel_event = env.process(manufacturing_process(env, 'Station Parallel', mean_times[5], std_devs[5], data, package_id, station_1_event))

            # Station 4 depends on Station 2, Station 3, and Station Parallel
            station_4_event = env.process(manufacturing_process(env, 'Station 4', mean_times[3], std_devs[3], data, package_id, env.all_of([station_2_event, station_3_event, station_parallel_event])))

            # Station 5
            env.process(manufacturing_process(env, 'Station 5', mean_times[4], std_devs[4], data, package_id, station_4_event))

# Read demand data from CSV
demand_data = pd.read_csv('demand.csv')

# Convert timestamps to datetime objects
demand_data['timestamp'] = pd.to_datetime(demand_data['timestamp'])

# Initialize data for results
results = []

# Define mean times and standard deviations for each station
mean_times = [10, 12, 8, 15, 10, 9]  # Example means for each station (including the new parallel station)
std_devs = [2, 3, 1, 4, 2, 2]        # Example standard deviations for each station (including the new parallel station)

# Simulate for each day
start_time = demand_data['timestamp'].min().normalize()  # Start from the first day
end_time = demand_data['timestamp'].max().normalize() + pd.Timedelta(days=1)

current_time = start_time

while current_time < end_time:
    next_time = current_time + pd.Timedelta(days=1)
    
    # Filter data for the current day (17:00 of the previous day to 16:59 of the current day)
    daily_data = demand_data[(demand_data['timestamp'] >= current_time + pd.Timedelta(hours=17)) & 
                             (demand_data['timestamp'] < next_time + pd.Timedelta(hours=17))]
    
    if not daily_data.empty:
        # Adjust timestamps to start from 0 for the simulation
        daily_data['timestamp'] = (daily_data['timestamp'] - (current_time + pd.Timedelta(hours=17))).dt.total_seconds() / 60

        # Initialize environment and data for the current day
        data = []
        env = simpy.Environment()

        # Start the arrival process for the current day
        env.process(arrival_process(env, daily_data, mean_times, std_devs, data))

        # Run the simulation for the current day
        env.run()

        # Add the results to the overall results
        for entry in data:
            entry['day'] = current_time.date()
            results.append(entry)
    
    current_time = next_time

# Convert to DataFrame
df = pd.DataFrame(results)

# Display DataFrame
print(df)

# Plot results
sns.lineplot(data=df, x='start_time', y='station', hue='package_id', marker='o')
plt.xlabel('Time (minutes from 17:00 previous day)')
plt.ylabel('Station')
plt.title('Manufacturing Process Simulation')
plt.show()


Empty DataFrame
Columns: []
Index: []


ValueError: Could not interpret value `start_time` for `x`. An entry with this name does not appear in `data`.